# NGLview - Usage notes

Author: Javier Cerezo (javier.cerezo@uam.es)</p>
Date: March 2021

This notebook reviews some hints to use `NGLview` module, with enphasis on representation of orbitals from cubefiles.

In [ ]:
# Import modules
import nglview as nv
import ase.io
# Load psi4 to compute orbitals
import psi4

In [ ]:
# Generate testing data files
## XYZ
geomxyz = '''2
HF geometry
F 0.0 0.0 -0.650
H 0.0 0.0  0.488
'''
fxyz = 'test.xyz'
open(fxyz,'w').write(geomxyz)

## PDB
geompdb = '''TITLE Testing file
ATOM      1 F    MOL     1       0.000   0.000  -0.650
ATOM      2 H    MOL     1       0.000   0.000   0.488
CONECT    1
CONECT    2
'''
fpdb = 'test.pdb'
open(fpdb,'w').write(geompdb)

## Single strucuture 

NGLview can read some structures directly from files. Some supported formats are `cube`, `pdb`, `gro`...

In [ ]:
view = nv.show_file(fpdb)
view

Note that `xyz` is not supported. Gives no error, but `view` has no atoms

In [ ]:
view = nv.show_file(fxyz)
view

In order to use `xyz` (or other generic fomats), we can use structure objects from popular projects such as `ASE` or `MDAnalysis`, which are supported in `NGLView`

In [ ]:
mol = ase.io.read(fxyz)
view = nv.show_ase(mol)
view

Actually, the `view_X()` calls are a shortcut to the general procedure to plot single structures:
 1. Create a `Structure` class with the structure from the file, `ASE` class...
 2. Create the `NGLWidget` with the `Structure` as argument
 
An `NGLWidget` object can be instantiated with filename (with structure or trajectory) or NGLView's `Structure` object (not with a structure object from other module)

In [ ]:
# Structure() is the base class. Load specific format with specific derived class
mol = ase.io.read(fxyz)
structure = nv.ASEStructure(mol)
view = nv.NGLWidget(structure)
view

In [ ]:
view = nv.NGLWidget()
view.add_component(fpdb)
view

In [ ]:
# If we want to use an external structure object, 
# use it to generate the NGLview structure object first
view = nv.NGLWidget()
mol = ase.io.read(fxyz)
structure = nv.ASEStructure(mol)
view.add_component(structure)
view

### Controlling visualization parameters

In [ ]:
# The parameters are stored in a dict. Some relevant ones to remove depth cue.
# Place the abave representation in a position where the strucutre is clipped
# and run the below parameter
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}

### Adding surfaces (orbitals)

In [ ]:
# Compute cubes with psi4:

## Generate input
psi4_inp = geomxyz.split('\n')
psi4_inp = '\n'.join(psi4_inp[2:])
psi4_inp = '0 1\n' + psi4_inp
## Set psi4 Molecule
psi4_mol = psi4.geometry(psi4_inp)
## Compute
E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
## Generate cubes
psi4.driver.p4util.cubeprop(wfn)
## Change name
%mv Psi_a_*_2-B1.cube Orbital_2-B1.cube
%mv Psi_a_*_7-A1.cube Orbital_7-A1.cube
%rm Psi_*cube
cubefile1 = 'Orbital_2-B1.cube'
cubefile2 = 'Orbital_7-A1.cube'

In [ ]:
# If we want to use an external structure object, 
# use it to generate the NGLview structure object first
# ASE can also read strcutre from cubes (to read surfaces
# from cubes with ASE there is a special function)
view = nv.NGLWidget()
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
mol = ase.io.read(cubefile1)
structure = nv.ASEStructure(mol)
view.add_component(structure)
view

In [ ]:
# To ensure that the orientation of the structure 
# is the same as that of the structure, it is better
# to load the structure also from the cubefile
view = nv.NGLWidget()
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
mol = ase.io.read(cubefile1)
structure = nv.ASEStructure(mol)
view.add_component(structure)

Surface data are loaded over a `NGLwidget` object as a new component. By default, cube data are loaded with parameters that are not convinient for usual visualization. In this case, clear the representation and add it again.

In [ ]:
# surface data on component_1
view.add_component(cubefile1)
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view

#### Adding multiple surfaces

This can be done in two ways:

 1. Removing component_1 and generate it for every surface
 2. Load all surfaces on separate compoents, which are hidden at first

**Option 1**

In [ ]:
# Structure
mol = ase.io.read(cubefile1)
view = nv.show_ase(mol)
# component_1: cubefile1
view.add_component(cubefile1)
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view

In [ ]:
# Change to surface on cubefile2
# component_1: cubefile2
c1 = view.component_1
view.remove_component(c1)
view.add_component(cubefile2)
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)

**Option 2**

In [ ]:
# Structure
mol = ase.io.read(cubefile1)
view = nv.show_ase(mol)
# component_1
view.add_component(cubefile1)
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_1.hide()
# component_2
view.add_component(cubefile2)
view.component_2.clear()
view.component_2.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_2.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_2.hide()

# View surface on cubefile1abs
view.component_1.show()
view

In [ ]:
# Change to surface on cubefile2
# Hide component_1 and show component_2
view.component_1.hide()
view.component_2.show()

Note that **Option 2** is much faster, with virtually no delay. So it is more convinient to use with interactive widgets (although it takes more memory)

Moreover, it is also convinient to cast all components as the are created to easily access them:

In [ ]:
# Structure
mol = ase.io.read(cubefile1)
view = nv.show_ase(mol)
# Get orbitals
orbs = []
# component_1
orbs.append(view.add_component(cubefile1))
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_1.hide()
# component_2
orbs.append(view.add_component(cubefile2))
view.component_2.clear()
view.component_2.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_2.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_2.hide()

# View surface on cubefile1abs
orbs[0].show()
view

In [ ]:
# Change to surface on cubefile2
# Hide component_1 and show component_2
orbs[0].hide()
orbs[1].show()

## Trajectories

Generate a trajectory from single structures

In [ ]:
# Testing geometries
geomxyz_1 = '''2
HF geometry
F 0.0 0.0 0
H 0.0 0.0 1.0
'''
fxyz_1 = 'test1.xyz'
open(fxyz_1,'w').write(geomxyz_1)
geomxyz_2 = '''2
HF geometry
F 0.0 0.0 0
H 0.0 0.0 1.5
'''
fxyz_2 = 'test2.xyz'
open(fxyz_2,'w').write(geomxyz_2)

In [ ]:
# Compute cubes with psi4:

## GEO 1 ##

## Generate input
psi4_inp = geomxyz_1.split('\n')
psi4_inp = '\n'.join(psi4_inp[2:])
psi4_inp = '0 1\n' + psi4_inp
## Set psi4 Molecule
psi4_mol = psi4.geometry(psi4_inp)
## Compute
E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
## Generate cubes
psi4.driver.p4util.cubeprop(wfn)
## Change name
%mv Psi_a_*_2-B1.cube Orbital1_2-B1.cube
%mv Psi_a_*_7-A1.cube Orbital1_7-A1.cube
%rm Psi_*cube
cubefile1_1 = 'Orbital1_2-B1.cube'
cubefile2_1 = 'Orbital1_7-A1.cube'

## GEO 2 ##

## Generate input
psi4_inp = geomxyz_2.split('\n')
psi4_inp = '\n'.join(psi4_inp[2:])
psi4_inp = '0 1\n' + psi4_inp
## Set psi4 Molecule
psi4_mol = psi4.geometry(psi4_inp)
## Compute
E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
## Generate cubes
psi4.driver.p4util.cubeprop(wfn)
## Change name
%mv Psi_a_*_2-B1.cube Orbital2_2-B1.cube
%mv Psi_a_*_7-A1.cube Orbital2_7-A1.cube
%rm Psi_*cube
cubefile1_2 = 'Orbital2_2-B1.cube'
cubefile2_2 = 'Orbital2_7-A1.cube'

print('Done')

In [ ]:
# Name of intermediate file
trfile='scan.traj'

# From single sctructures to trajectory loaded on nglview
## Get structures into ase object
mol_1 = ase.io.read(cubefile1_1)
mol_2 = ase.io.read(cubefile1_2)
mols = [mol_1, mol_2]
## Write into traj file
ase.io.write(trfile,mols)
## Read trajectory intp ase and convert to nglview object
asetraj = ase.io.trajectory.TrajectoryReader(trfile)
trajectory = nv.ASETrajectory(asetraj)
## Generate view and load trajectory
#view = nv.NGLWidget()
#view.add_component(trajectory)
## Or use the shortcut
view = nv.show_asetraj(asetraj)
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}

In [ ]:
# Check number of frames
len(asetraj)

In [ ]:
# The viewer for traj includes controls
view
# Can be disable with 
#view.player.widget_player_slider.close()
# But needs to be done before calling view

When loading a trajectory, the `NGLwidget` contains controls to manage the trajectory. This can also be done manipulating the wiew (if we need to use external widgets)

In [ ]:
orbs = []
# component_1 (store component address in orbs list)
orbs.append(view.add_component(cubefile2_1))
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_1.hide()
# component_2
orbs.append(view.add_component(cubefile2_2))
view.component_2.clear()
view.component_2.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_2.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_2.hide()

In [ ]:
# Show surface for first geom
view.frame = 0
orbs[0].show()

In [ ]:
# Show surface for second geom
view.frame = 1
orbs[0].hide()
orbs[1].show()

## Interactivity: integration with ipywidgets

It is likely that event handling can be access through some attributes of `NGLwiget` object (`view`), e.g.: </p>
`view.notify_change()`</p>
`view.trait_events()`</p>
...

This is to be explored (e.g., it is likely that the change from the inserted controls can be linked to an update of the showed component)

In the meantime (while not knowing how to do this), we can use `interact()` within widgets, together with `view.frame` and the list of components in `orbs`. The list can rebuild for each orbital in order to avoid having the all in memory, and only having the "fast" update along trajectory frames

In [ ]:
# Import ipywidgets (and some specific to the main namespace)
import ipywidgets as widgets
from ipywidgets import interactive, VBox, HBox, Output

# Import matplotlib and use magic
%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
# Testing geometries
geomxyz_1 = '''2
HF geometry
F 0.0 0.0 0
H 0.0 0.0 1.0
'''
fxyz_1 = 'test1.xyz'
open(fxyz_1,'w').write(geomxyz_1)
geomxyz_2 = '''2
HF geometry
F 0.0 0.0 0
H 0.0 0.0 1.5
'''
fxyz_2 = 'test2.xyz'
open(fxyz_2,'w').write(geomxyz_2)

In [ ]:
## PSI4 calculation ##
# Compute energies with psi4 and store orbital energies

orb_eners=[] #orb_eners[geo][symm][n_in_symm]
mol_eners=[] #mol_eners[geo]
cubefile = [[None,None],[None,None]] #cubefile[geo][mo]
## GEO 1 ##

## Generate input
psi4_inp = geomxyz_1.split('\n')
psi4_inp = '\n'.join(psi4_inp[2:])
psi4_inp = '0 1\n' + psi4_inp
## Set psi4 Molecule
psi4_mol = psi4.geometry(psi4_inp)
## Compute
E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
## Generate cubes
psi4.driver.p4util.cubeprop(wfn)
## Change name
%mv Psi_a_*_2-B1.cube Orbital1_2-B1.cube
%mv Psi_a_*_7-A1.cube Orbital1_7-A1.cube
%rm Psi_*cube
cubefile[0][0] = 'Orbital1_2-B1.cube'
cubefile[0][1] = 'Orbital1_7-A1.cube'
# Store orb energies (per symm)
orb_eners.append(wfn.epsilon_a().nph)
# Store total energy
mol_eners.append(E)

## GEO 2 ##

## Generate input
psi4_inp = geomxyz_2.split('\n')
psi4_inp = '\n'.join(psi4_inp[2:])
psi4_inp = '0 1\n' + psi4_inp
## Set psi4 Molecule
psi4_mol = psi4.geometry(psi4_inp)
## Compute
E, wfn = psi4.energy('b3lyp/6-31G(d)',return_wfn=True)
## Generate cubes
psi4.driver.p4util.cubeprop(wfn)
## Change name
%mv Psi_a_*_2-B1.cube Orbital2_2-B1.cube
%mv Psi_a_*_7-A1.cube Orbital2_7-A1.cube
%rm Psi_*cube
cubefile[1][0] = 'Orbital2_2-B1.cube'
cubefile[1][1] = 'Orbital2_7-A1.cube'
# Store orb energies (per symm)
orb_eners.append(wfn.epsilon_a().nph)
# Store total energy
mol_eners.append(E)

# Convinient wraper of orb energies
# cube_eners[geo][mo]
cube_eners=[[orb_eners[0][2][1],orb_eners[0][0][6]],[orb_eners[1][2][1],orb_eners[1][0][6]]]

print('Done')

### Interactive geometry selection

In [ ]:
# Set traj view
## Name of intermediate file
trfile='scan.traj'
# From single sctructures to trajectory loaded on nglview
## Get structures into ase object
mol_1 = ase.io.read(cubefile1_1)
mol_2 = ase.io.read(cubefile1_2)
mols = [mol_1, mol_2]
## Write into traj file
ase.io.write(trfile,mols)
## Read trajectory intp ase and convert to nglview object
asetraj = ase.io.trajectory.TrajectoryReader(trfile)
## Generate view with trajectory
view = nv.show_asetraj(asetraj)
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
# Disable embeded player slider
view.player.widget_player_slider.close()

In [ ]:
# Store orbs
orbs = []
# component_1 (store component address in orbs list)
orbs.append(view.add_component(cubefile2_1))
view.component_1.clear()
view.component_1.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_1.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_1.hide()
# component_2
orbs.append(view.add_component(cubefile2_2))
view.component_2.clear()
view.component_2.add_surface(opacity=0.5, color='blue', isolevel=1. )
view.component_2.add_surface(opacity=0.5, color='red', isolevel=-1.)
view.component_2.hide()

In [ ]:
def update_geom(i):
    global view, orbs
    # Geom
    view.frame = i
    # Orbs
    for orb in orbs:
        orb.hide()
    orbs[i].show()
    
controls = interactive(update_geom, 
                       i=(0,1,1))
VBox([view, controls])

### Add orbital diagram

Use `matplotlib` to plot MO diagram. The figure is placed in a `Output` widget so that it can be handled arranged with `HBox` and `VBox`

In [ ]:
# WARNING: figures must be closed
out_diagram = Output(layout={'border': '1px solid black'})
with out_diagram:
    OMdiagram, ax = plt.subplots(1,figsize=(2,6))
    OMdiagram.tight_layout()
    OMdiagram.canvas.header_visible = False
    ax.set_ylabel('Energy, a.u.')
    ax.set_xticklabels([])

The view object must be called only once for display after being created. It is therefore convinient to put the creation and display in the same cell

In [ ]:
# Set traj view
## Name of intermediate file
trfile='scan.traj'
# From single sctructures to trajectory loaded on nglview
## Get structures into ase object
mol_1 = ase.io.read(cubefile[0][0])
mol_2 = ase.io.read(cubefile[1][0])
mols = [mol_1, mol_2]
## Write into traj file
ase.io.write(trfile,mols)
## Read trajectory intp ase and convert to nglview object
asetraj = ase.io.trajectory.TrajectoryReader(trfile)
## Generate view with trajectory
view = nv.show_asetraj(asetraj)
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
# Disable embeded player slider
view.player.widget_player_slider.close()

# Store orbs surfare representations
orbs = []
# component_1 (store component address in orbs list)
orbs.append(view.add_component(cubefile[0][1]))
orbs[-1].clear()
orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
orbs[-1].hide()
# component_2
orbs.append(view.add_component(cubefile[1][1]))
orbs[-1].clear()
orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
orbs[-1].hide()

# Build an output container to print info about orbital
out_label=Output()

# DISPLAY
def update_geom(i):
    global view, orbs
    # Geom
    view.frame = i
    # Orbs
    for orb in orbs:
        orb.hide()
    orbs[i].show()
    ax.clear()
    ax.hlines(cube_eners[i],xmin=1,xmax=2,color='k')
    
controls = interactive(update_geom, 
                       i=widgets.IntSlider(value=0,min=0,max=1,description='Geom'))
surfbox = VBox([out_label,view, controls],layout={'width': '700px'})
HBox([surfbox,out_diagram])

### Interactive MO selection

In [ ]:
# Initialize orb/geo ids
mo_  = 1
geo_ = 0


# Set traj view
## Name of intermediate file
trfile='scan.traj'
# From single sctructures to trajectory loaded on nglview
## Get structures into ase object
mol_1 = ase.io.read(cubefile[0][0])
mol_2 = ase.io.read(cubefile[1][0])
mols = [mol_1, mol_2]
## Write into traj file
ase.io.write(trfile,mols)
## Read trajectory intp ase and convert to nglview object
asetraj = ase.io.trajectory.TrajectoryReader(trfile)
## Generate view with trajectory
view = nv.show_asetraj(asetraj)
view.parameters = {"clipNear": 0, "clipFar": 100, "clipDist": 1}
# Disable embeded player slider
view.player.widget_player_slider.close()

# Store orbs surfare representations
orbs = []
# component_1 (store component address in orbs list)
orbs.append(view.add_component(cubefile[0][mo_]))
orbs[-1].clear()
orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
orbs[-1].hide()
# component_2
orbs.append(view.add_component(cubefile[1][mo_]))
orbs[-1].clear()
orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
orbs[-1].hide()

# Build an output container to print info about orbital
out_label=Output()

# DISPLAY
    
def update_repr(geo,mo):
    global view, orbs, mo_, geo_
    
    out_label.clear_output()
    with out_label:
        print('Geom:',geo,geo_)
        print('MO:',mo,mo_)
      
    # Only one MO is loaded in memory (for all geoms)
    # So, if it changes, reload the right one
    if (mo != mo_):
        #Update mo (load new set)
        # remove current
        for orb in orbs:
            view.remove_component(orb)
        # load new mo
        orbs=[]
        # component_1 (store component address in orbs list)
        orbs.append(view.add_component(cubefile[0][mo]))
        orbs[-1].clear()
        orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
        orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
        orbs[-1].hide()
        # component_2
        orbs.append(view.add_component(cubefile[1][mo]))
        orbs[-1].clear()
        orbs[-1].add_surface(opacity=0.5, color='blue', isolevel=1. )
        orbs[-1].add_surface(opacity=0.5, color='red', isolevel=-1.)
        orbs[-1].hide()

    #Update geo
    # Geom
    view.frame = geo
    # Orbs
    orbs[geo_].hide()
        
    # Display selected
    orbs[geo].show()
    
    # Update diagram
    ax.clear()
    ax.hlines(cube_eners[geo],xmin=1,xmax=2,color='k')
    ax.hlines(cube_eners[geo][mo],xmin=1,xmax=2,linewidths=3,color='yellow', visible=True, alpha=0.7)
    
    # Update ids
    mo_=mo
    geo_=geo
    
    
# DISPLAY
controls = interactive(update_repr, 
                       geo=widgets.IntSlider(value=0,min=0,max=1,description='Geom'),
                       mo=widgets.IntSlider(value=1,min=0,max=1,description='MO'))
surfbox = VBox([out_label,view, controls],layout={'width': '700px'})
HBox([surfbox,out_diagram])